# Sage Special Topics: Cython

## Overview

[Cython](https://cython.org/) is a compiler  
for the Python programming language along with  
the extended Cython programming language.  
This combination allows you to enjoy  
the readability of Python and 
the efficiency of C.

This means you may:  
- use the **static data type in C** to speed up the program,  
- **wrap C functions** into Python interface, and  
- yet more features that I don't totally understand.

Sage developing team plays an important role  
in building up Cython; see [the history of Cython](https://en.wikipedia.org/wiki/Cython#History).  
However, Cython is not limited to the Sage environment,  
and this tutorial is also applicable for general Python users.

## Installation

If you are using a Sage notebook,  
or a Jupyter notebook under the Sage kernel,  
you may ignore this section.

For general Python users,  
you may use  
```Python
pip install Cython --user
```
to **install** Cython on your machine.  

To use it in a Jupyter notebook under the Python kernel,  
you have to run the following cell  
to **activate** the Cython extension.

In [1]:
%load_ext Cython

For using Cython in the console or other IDEs,  
please refer to the [Workflow](#Workflow) section.

## Workflow

As mentioned before,  
Cython is a **compiler** rather than an interpreter,  
so each piece of the code need to be compiled before execution.

Here is the basic workflow: (See [Basic Tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) of the official Cython documentation for more details.)  
- Prepare a `cython_code.pyx` file that contains your Cython code.
- Prepare a `setup.py` file that contains the setup information.
- run `python setup.py build_ext --inplace` in the terminal  
to generate `cython_code.so` (unix) or `cython_code.pyd` (Windows).
- In another Python file or in a Jupyter notebook, `import cython_code`.

Sample `cython_code.pyx`:

In [ ]:
from cpython.version cimport PY_VERSION_HEX

def print_cython_version():
    print(PY_VERSION_HE)

Sample `setup.py`:

In [ ]:
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cython_code.pyx")
)

Sample usage:

In [ ]:
import cython_code

## Examples

In [3]:
def a_prime(p):
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

def primes_below(n):
    primes = [p for p in range(2,n+1) if a_prime(p)]
    return primes

In [4]:
%%timeit
primes = primes_below(10000)

1 loop, best of 3: 230 ms per loop


In [5]:
%%cython

def a_prime_pinc(p):
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

def primes_below_pinc(n):
    primes = [p for p in range(2,n+1) if a_prime_pinc(p)]
    return primes

In [6]:
%%timeit
primes = primes_below_pinc(10000)

10 loops, best of 3: 148 ms per loop


In [119]:
%%cython

cdef bint a_prime_c(int p):
    cdef int i
    for i in range(2,p):
        if p % i == 0:
            return False
    else:
        return True

cpdef primes_below_c(int n):
    primes = [p for p in range(2,n+1) if a_prime_c(p)]
    return primes

In [120]:
%%timeit
primes = primes_below_c(10000)

100 loops, best of 5: 10.2 ms per loop


In [141]:
%%cython
# distutils: language=c++

from libcpp.vector cimport vector

def cprimes(unsigned int nb_primes):
    cdef int n, i
    cdef vector[int] p
    p.reserve(nb_primes)  # allocate memory for 'nb_primes' elements.

    n = 2
    while p.size() < nb_primes:  # size() for vectors is similar to len()
        for i in p:
            if n % i == 0:
                break
        else:
            p.push_back(n)  # push_back is similar to append()
        n += 1

    # Vectors are automatically converted to Python
    # lists when converted to Python objects.
    return p

# print(cprimes(1229))

In [142]:
%%timeit
cprimes(1229)

100 loops, best of 5: 2.11 ms per loop


In [138]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
